<a href="https://colab.research.google.com/github/Pradeepjha88/Cardiac_attack_prediction/blob/master/Customer_Segmentation_Online_Retail.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

#### Dataset source
https://archive.ics.uci.edu/ml/datasets/online+retail

In [ ]:
df1 = pd.read_excel("Online_Retail.xlsx")
df1.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


We need to do santity check about the data

In [ ]:
df1.Country.nunique()

38

In [ ]:
df1.Country.unique()

array(['United Kingdom', 'France', 'Australia', 'Netherlands', 'Germany',
       'Norway', 'EIRE', 'Switzerland', 'Spain', 'Poland', 'Portugal',
       'Italy', 'Belgium', 'Lithuania', 'Japan', 'Iceland',
       'Channel Islands', 'Denmark', 'Cyprus', 'Sweden', 'Austria',
       'Israel', 'Finland', 'Bahrain', 'Greece', 'Hong Kong', 'Singapore',
       'Lebanon', 'United Arab Emirates', 'Saudi Arabia',
       'Czech Republic', 'Canada', 'Unspecified', 'Brazil', 'USA',
       'European Community', 'Malta', 'RSA'], dtype=object)

In [ ]:
customer_country=df1[['Country','CustomerID']].drop_duplicates()

x = customer_country.groupby(['Country'])['CustomerID'].aggregate('count')/customer_country.shape[0] * 100
x.reset_index().sort_values('CustomerID', ascending=False)

,Country,CustomerID
36,United Kingdom,89.997722
14,Germany,2.164502
13,France,1.982228
31,Spain,0.706311
3,Belgium,0.569606
33,Switzerland,0.478469
27,Portugal,0.432900
19,Italy,0.341763
12,Finland,0.273411
1,Austria,0.250627


More than 90% of the customers in the data are from United Kingdom, SO we are interested only in UK data.

In [ ]:
df1 = df1.loc[df1['Country'] == 'United Kingdom']

In [ ]:
df1.isnull().sum()/df1.shape[0] * 100

InvoiceNo       0.000000
StockCode       0.000000
Description     0.293454
Quantity        0.000000
InvoiceDate     0.000000
UnitPrice       0.000000
CustomerID     26.963861
Country         0.000000
dtype: float64

In [ ]:
df1 = df1[df1['CustomerID'].isnull() == False] #pd.notnull
df1.isnull().sum(axis=0)

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

Check the min and max values in Unit price column

In [ ]:
df1.describe()

,Quantity,UnitPrice,CustomerID
count,361878.000000,361878.000000,361878.000000
mean,11.077029,3.256007,15547.871368
std,263.129266,70.654731,1594.402590
min,-80995.000000,0.000000,12346.000000
25%,2.000000,1.250000,14194.000000
50%,4.000000,1.950000,15514.000000
75%,12.000000,3.750000,16931.000000
max,80995.000000,38970.000000,18287.000000


Remove the negative values in Quantity column

In [ ]:
df1 = df1[(df1['Quantity']>0)]
df1.Quantity.min()

1

After cleaning up, we now dealing with 354345 rows and 8 columns

In [ ]:
df1.shape

(354345, 8)

In [ ]:
# df1.to_csv("Transactions_data.csv")

Check unique value for each column

In [ ]:
df1.nunique()

InvoiceNo      16649
StockCode       3645
Description     3844
Quantity         294
InvoiceDate    15615
UnitPrice        403
CustomerID      3921
Country            1
dtype: int64

In [ ]:
df1.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


Add a column for total price

In [ ]:
df1['TotalPrice'] = df1['Quantity'] * df1['UnitPrice']
df1.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34


# Analysis of the Data

We want to find most imp customers

Customers which are frequent

Customers which are frequent recently

Customers which are frequent recently and also spend a good amount (monetory value)

### SO :: Frequency, Recency and Monetory Value all are important


Find out first and last order date in the data

In [ ]:
df1['InvoiceDate'].min()

Timestamp('2010-12-01 08:26:00')

In [ ]:
df1['InvoiceDate'].max()

Timestamp('2011-12-09 12:49:00')

Since recency is calculated for a point in time. The last invoice date is 2011-12-09, this is the date we will use to calculate recency.

In [ ]:
import datetime as dt
NOW = dt.datetime(2011,12,10)

In [ ]:
df1['InvoiceDate'] = pd.to_datetime(df1['InvoiceDate'])

In [ ]:
NOW = max(df1['InvoiceDate'])
NOW

Timestamp('2011-12-09 12:49:00')

# Create a RFM table

### 'recency', 'frequency' , 'monetary_value'

In [ ]:
rfmTable = df1.groupby('CustomerID').agg({'InvoiceDate': lambda x: (NOW - x.max()).days, # Recency
                                        'InvoiceNo': lambda x: len(x),      # Frequency
                                        'TotalPrice': lambda x: x.sum()}) # Monetary Value

rfmTable['InvoiceDate'] = rfmTable['InvoiceDate'].astype(int)

In [ ]:
def recency_calc(x):
  return (NOW - x.max()).days

def freq_calc(x):
  return len(x)

def monetary_price_calc(x):
  return x.sum()

In [ ]:
rfmTable = df1.groupby('CustomerID').agg({'InvoiceDate': recency_calc, # Recency
                                        'InvoiceNo': freq_calc,      # Frequency
                                        'TotalPrice': monetary_price_calc}) # Monetary Value

rfmTable['InvoiceDate'] = rfmTable['InvoiceDate'].astype(int)

In [ ]:
rfmTable.columns

Index(['InvoiceDate', 'InvoiceNo', 'TotalPrice'], dtype='object')

In [ ]:
#rename function
rfmTable.columns = ['recency', 'frequency' , 'monetary_value']

Calculate RFM metrics for each customer

In [ ]:
rfmTable.head()

,recency,frequency,monetary_value
CustomerID,,,
12346.0,325,1,77183.60
12747.0,1,103,4196.01
12748.0,0,4596,33719.73
12749.0,3,199,4090.88
12820.0,2,59,942.34


#### 10 Customers with highest Monetory values

In [ ]:
rfmTable.sort_values(['monetary_value'],ascending = False).iloc[:10,:]

,recency,frequency,monetary_value
CustomerID,,,
18102.0,0,431,259657.30
17450.0,7,337,194550.79
16446.0,0,3,168472.50
17511.0,2,963,91062.38
16029.0,38,242,81024.84
12346.0,325,1,77183.60
16684.0,3,277,66653.56
14096.0,3,5111,65164.79
13694.0,3,568,65039.62


### Customers with maximum manitory ratio

Monitory value per visit

In [ ]:
rfmTable['m_ratio'] = rfmTable['monetary_value'] / rfmTable['frequency']

In [ ]:
rfmTable.sort_values(['m_ratio'],ascending = False).iloc[:10,:]

,recency,frequency,monetary_value,m_ratio
CustomerID,,,,
12346.0,325,1,77183.60,77183.600000
16446.0,0,3,168472.50,56157.500000
15098.0,181,3,39916.50,13305.500000
15749.0,234,10,44534.30,4453.430000
15195.0,2,1,3861.00,3861.000000
13135.0,196,1,3096.00,3096.000000
17846.0,84,1,2033.10,2033.100000
18087.0,97,2,4055.72,2027.860000
16532.0,17,4,6748.80,1687.200000


##### NOTE :: ONLY sorting will not work here

In [ ]:
rfmTable.sort_values(['frequency', 'monetary_value'], ascending=[True, False])

,recency,frequency,monetary_value,m_ratio
CustomerID,,,,
12346.0,325,1,77183.60,77183.600000
15195.0,2,1,3861.00,3861.000000
13135.0,196,1,3096.00,3096.000000
17846.0,85,1,2033.10,2033.100000
18133.0,212,1,931.50,931.500000
...,...,...,...,...
14606.0,1,2700,12156.65,4.502463
12748.0,0,4596,33719.73,7.336756
14096.0,4,5111,65164.79,12.749910


In [ ]:
(NOW - dt.datetime(2011,1,18)).days==326

True

The easies way to split metrics into segments is by using quartile. 

1. This gives us a starting point for detailed analysis
2. 4 segments are easy to understand and explain

In [ ]:
rfmTable.describe()

,recency,frequency,monetary_value,m_ratio
count,3921.000000,3921.000000,3921.000000,3921.000000
mean,91.188472,90.371079,1863.910113,71.738338
std,99.528995,217.796155,7481.922217,1543.783600
min,0.000000,1.000000,0.000000,0.000000
25%,17.000000,17.000000,300.040000,11.175000
50%,50.000000,41.000000,651.820000,17.284211
75%,142.000000,99.000000,1575.890000,24.393182
max,373.000000,7847.000000,259657.300000,77183.600000


In [ ]:
quantiles = rfmTable.quantile(q=[0.05,0.25,0.5,0.75,0.95])
quantiles

,recency,frequency,monetary_value,m_ratio
0.05,2.0,4.0,110.75,4.420623
0.25,17.0,17.0,300.04,11.175000
0.50,50.0,41.0,651.82,17.284211
0.75,142.0,99.0,1575.89,24.393182
0.95,310.0,314.0,5493.46,93.500000


In [ ]:
quantiles = quantiles.to_dict()
quantiles

{'frequency': {0.05: 4.0, 0.25: 17.0, 0.5: 41.0, 0.75: 99.0, 0.95: 314.0},
 'm_ratio': {0.05: 4.420623376623377,
  0.25: 11.174999999999999,
  0.5: 17.284210526315793,
  0.75: 24.393181818181812,
  0.95: 93.5},
 'monetary_value': {0.05: 110.75,
  0.25: 300.03999999999996,
  0.5: 651.8199999999999,
  0.75: 1575.89,
  0.95: 5493.46},
 'recency': {0.05: 2.0, 0.25: 17.0, 0.5: 50.0, 0.75: 142.0, 0.95: 310.0}}

Create a segmented RFM table

In [ ]:
segmented_rfm = rfmTable

Lowest recency, highest frequency and monetary are our best customers 

# Tree Based Segmentation

In [ ]:
rfmTable[ (rfmTable['recency'] < 50) & (rfmTable['frequency'] > 10) ]

,recency,frequency,monetary_value,m_ratio
CustomerID,,,,
12747.0,1,103,4196.01,40.737961
12748.0,0,4596,33719.73,7.336756
12749.0,3,199,4090.88,20.557186
12820.0,2,59,942.34,15.971864
12826.0,2,91,1474.72,16.205714
...,...,...,...,...
18274.0,29,11,175.92,15.992727
18276.0,43,14,335.86,23.990000
18282.0,7,12,178.05,14.837500


In [ ]:
max_quant = df1['Quantity'].max()
mask = (df1['Country'] == 'United Kingdom') & (df1['Quantity'] > 100) & (df1['TotalPrice'] < 1000 )
df1[mask].shape

(3030, 9)

### Segment Customers on Recency using Quantiles

In [ ]:
def RScore_simple(x): # Simple recency score function with fixed values
    if x <= 17.0:
        return 1
    elif x <= 50.0:
        return 2
    elif x <= 141.5: 
        return 3
    else:
        return 4

In [ ]:
def RScore(x,p,d): # Recency score segmentation by using quantile 
    if x <= d[p][0.25]:#17.0
        return 1
    elif x <= d[p][0.50]:#50.0
        return 2
    elif x <= d[p][0.75]: 
        return 3
    else:
        return 4
    
def FMScore(x,p,d): # Single function to segment frequency and monetory value by parameters
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.50]:
        return 3
    elif x <= d[p][0.75]: 
        return 2
    else:
        return 1

In [ ]:
segmented_rfm['r_quartile'] = segmented_rfm['recency'].apply(RScore_simple)#, args=('recency',quantiles,))
segmented_rfm['f_quartile'] = segmented_rfm['frequency'].apply(FMScore, args=('frequency',quantiles,))
segmented_rfm['m_quartile'] = segmented_rfm['monetary_value'].apply(FMScore, args=('monetary_value',quantiles,))

Add segment numbers to the RFM table

In [ ]:
segmented_rfm.head()

,recency,frequency,monetary_value,m_ratio,r_quartile,f_quartile,m_quartile
CustomerID,,,,,,,
12346.0,325,1,77183.60,77183.600000,4,4,1
12747.0,1,103,4196.01,40.737961,1,1,1
12748.0,0,4596,33719.73,7.336756,1,1,1
12749.0,3,199,4090.88,20.557186,1,1,1
12820.0,2,59,942.34,15.971864,1,2,2


RFM segments split your customer base into an imaginary 3D cube. It is hard to visualize. However, we can sort it out.

Add a new column to combine RFM score, 111 is the highest score as we determined earlier.

In [ ]:
segmented_rfm['RFMScore'] = segmented_rfm.r_quartile.map(str) \
                            + segmented_rfm.f_quartile.map(str) \
                            + segmented_rfm.m_quartile.map(str)
segmented_rfm.head()

,recency,frequency,monetary_value,m_ratio,r_quartile,f_quartile,m_quartile,RFMScore
CustomerID,,,,,,,,
12346.0,325,1,77183.60,77183.600000,4,4,1,441
12747.0,1,103,4196.01,40.737961,1,1,1,111
12748.0,0,4596,33719.73,7.336756,1,1,1,111
12749.0,3,199,4090.88,20.557186,1,1,1,111
12820.0,2,59,942.34,15.971864,1,2,2,122


Apparently, the first customer is not our best customer at all.

Here is top 10 of our best customers!

In [ ]:
segmented_rfm[segmented_rfm['RFMScore']=='111'].sort_values('monetary_value', ascending=False).head(10)

,recency,frequency,monetary_value,m_ratio,r_quartile,f_quartile,m_quartile,RFMScore
CustomerID,,,,,,,,
18102.0,0,431,259657.30,602.453132,1,1,1,111
17450.0,7,337,194550.79,577.302047,1,1,1,111
17511.0,2,963,91062.38,94.561142,1,1,1,111
16684.0,3,277,66653.56,240.626570,1,1,1,111
14096.0,3,5111,65164.79,12.749910,1,1,1,111
13694.0,3,568,65039.62,114.506373,1,1,1,111
15311.0,0,2379,60767.90,25.543464,1,1,1,111
13089.0,2,1818,58825.83,32.357442,1,1,1,111
15769.0,6,130,56252.72,432.713231,1,1,1,111


# Discussion:

- Can only Monetory value make customer IMP ?

- Can Frequency make a customer IMP?

- Any other parameters to find IMP customers ?



# NEXT LEVEL

- how to use prediction model to predict monetory value ?

- What can be hypothesis testing applied here?
